In [1]:
import numpy as np
import torch
#import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/akshatgu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)
CUDA_LAUNCH_BLOCKING=1

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is False
0


/home/akshatgu/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'eos':1, 'unk': 2}#Padding indx = 0, eos = 1, unkown_idx = 2, indexing starts from 3
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 3
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                if len(utterance) != 0:
                    utterance_to_idx = []

                    for phone in utterance:
                        if phone not in phone_to_idx:
                            phone = 'unk'

                        utterance_to_idx.append(phone_to_idx[phone])

                    self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.tensor(y_lst)

    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [8]:
class MyLMDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                if len(utterance) != 0:
                    utterance_to_idx = []

                    for phone in utterance:
                        if phone not in phone_to_idx:
                            phone = 'unk'

                        utterance_to_idx.append(phone_to_idx[phone])

                    self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        output_vector = self.all_data[index][0][1:] + [1]

        return input_vector, output_vector

In [9]:
def collate_LM(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]
    

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    y = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)
        y[i,:len(x_np)] = torch.tensor(y_lst[i])


    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [10]:
def get_intents():
    all_intents = ['1', '2', '3', '4', '5', '6']
    return all_intents

def get_intent_labels(class_type):
    all_intents = get_intents()
        
    intent_labels = {}
    labels_to_intents = {}
    for i, intent in enumerate(all_intents):
        intent_labels[intent] = i
        labels_to_intents[i] = intent
        
    return intent_labels, labels_to_intents

In [15]:
class_type = 'intents'

intent_labels, labels_to_intents = get_intent_labels(class_type)

#Loading data
split = '1'
train_file = '../../Tamil_Dataset/datasplit_top5_split1/tamil_train_split_' + split + '.pkl'
test_file = '../../Tamil_Dataset/datasplit_top5_split1/tamil_test_split_' + split + '.pkl'
#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
vocab_size = len(phone_to_idx) + 1
print(vocab_size)

48


In [16]:
train_dataset = MyLMDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader_LM = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_LM)

In [17]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [18]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=128, label_size=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        #self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(embed_size*2)

        self.lstm_LM = nn.LSTM(embed_size, hidden_size, num_layers=1)
        self.lstm = nn.LSTM(embed_size*2, hidden_size, num_layers=1)

        self.linear_LM = nn.Linear(hidden_size, vocab_size)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths, lm = True):
        """
        padded_x: (B,T) padded LongTensor
        """
        # B,T,H
        
        input = self.embed(x)
        
        if lm:
            pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
            output, (hn, cn) = self.lstm_LM(pack_tensor)
            output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
            logits = self.linear_LM(output)

        else:

            # (B,T,H) -> (B,H,T)
            input = input.transpose(1,2)

            #cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)
            cnn_output = torch.cat([self.cnn(input), self.cnn2(input)], dim=1)

            # (B,H,T)
            input = F.relu(self.batchnorm(cnn_output))

            input = input.transpose(1,2)

            pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
            output, (hn, cn) = self.lstm(pack_tensor)

            logits = self.linear(hn[0])

        return logits

In [21]:
model = RNNClassifier(vocab_size)
opt = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
#opt = SGD(model.parameters(), lr=0.01)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(48, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm_LM): LSTM(128, 128)
  (lstm): LSTM(256, 128)
  (linear_LM): Linear(in_features=128, out_features=48, bias=True)
  (linear): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
#Train language model
print(class_type, split)
max_acc = 0

for j in range(100):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader_LM):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()

        logits = model(x, lengths)
        
        loss = criterion(logits.permute(0,2,1), y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()
                    

    print(j, " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

intents 1
0  train loss:  3.799136781692505 --time: 7.019295692443848
1  train loss:  3.7163057804107664 --time: 6.627717971801758
2  train loss:  3.6073073863983156 --time: 6.895938873291016
3  train loss:  3.5152974128723145 --time: 6.914318323135376
4  train loss:  3.439412784576416 --time: 6.677660226821899
5  train loss:  3.3737284660339357 --time: 6.713462591171265
6  train loss:  3.337066411972046 --time: 6.815105438232422
7  train loss:  3.2707120895385744 --time: 6.735300779342651
8  train loss:  3.248623561859131 --time: 6.979761600494385
9  train loss:  3.1365509986877442 --time: 6.45695686340332
10  train loss:  3.1491559982299804 --time: 6.8487372398376465
11  train loss:  3.134732246398926 --time: 7.06435751914978
12  train loss:  3.0490954875946046 --time: 6.676224946975708
13  train loss:  3.0754759311676025 --time: 7.0568459033966064
14  train loss:  3.029900312423706 --time: 6.867817640304565
15  train loss:  2.996788167953491 --time: 6.82466459274292
16  train loss: 

In [20]:
print(class_type, split)
max_acc = 0

for j in range(200):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()

        logits = model(x, lengths, False)

        loss = criterion(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
                    

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:
        
        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths, False)

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y
    
        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print(j, "validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

intents 1
train acc:  0.275  train loss:  1.7683594226837158 --time: 7.76800012588501
0 validation:  0.2875 --max 0.2875 --time: 8.925299406051636
train acc:  0.290625  train loss:  1.658125400543213 --time: 7.5629355907440186
1 validation:  0.3 --max 0.3 --time: 8.695011615753174
train acc:  0.44375  train loss:  1.524280834197998 --time: 7.5245020389556885
2 validation:  0.325 --max 0.325 --time: 8.66071605682373
train acc:  0.503125  train loss:  1.3635321855545044 --time: 7.373891353607178
3 validation:  0.275 --max 0.325 --time: 8.508850574493408
train acc:  0.5625  train loss:  1.2472010850906372 --time: 6.863156795501709
4 validation:  0.3375 --max 0.3375 --time: 7.989182233810425
train acc:  0.61875  train loss:  1.0846196293830872 --time: 7.188095808029175
5 validation:  0.3375 --max 0.3375 --time: 8.317760705947876
train acc:  0.7125  train loss:  0.9457153797149658 --time: 7.1948912143707275
6 validation:  0.3125 --max 0.3375 --time: 8.331063032150269
train acc:  0.771875  t

KeyboardInterrupt: 

In [ ]:
1 layer, 512, only 2 CNN contexts